In [ ]:
# Get all heat pumps (air/ground source)
hp_links = n.links[n.links.carrier.str.contains("heat_pump")]

# Find their actual power consumption in MWh over time
hp_electricity_usage = n.links_t.p0[hp_links.index]
# Get all generators and their output
gen_output = n.generators_t.p

# Map each generator to its bus
gen_buses = n.generators.bus

# Loop over each region (or bus) and see what powered the heat pumps
for bus in n.buses.index:
    hp_names = [name for name in hp_links.index if name.startswith(bus)]
    if not hp_names:
        continue

    # Total electricity for heat pumps in this bus
    total_hp_input = hp_electricity_usage[hp_names].sum(axis=1)

    # All generators on this bus
    gen_names = gen_buses[gen_buses == bus].index
    gen_contrib = gen_output[gen_names]

    # Normalize their share of total generation
    gen_share = gen_contrib.div(gen_contrib.sum(axis=1), axis=0)

    # Estimate how much each generator powered heat pumps
    fuel_contribution = gen_share.mul(total_hp_input, axis=0)

    print(f"Region {bus}:")
    print(fuel_contribution.sum())


In [ ]:
import matplotlib.pyplot as plt

# Sum over all hours to get annual contributions
total_contribution = fuel_contribution.sum()

# Pie chart
total_contribution.plot(kind='pie', autopct='%1.1f%%', startangle=99)
plt.title(f"Fuel mix for heat pumps at {bus}")
plt.ylabel("")
plt.tight_layout()
plt.show()
